In [1]:
import sys

sys.path.append("../src/stong_core_cython")

In [2]:
import gaussian_overlap
import importlib
import numpy as np
from scipy.special import factorial2 as fact2

importlib.reload(gaussian_overlap)
from gaussian_overlap import *

In [3]:
center_a = np.array([0.0, 0.0, 0.0])
lmn_a = (2, 1, 1)
exps_a = np.array([1.0, 2.0, 3.0])
coefs_a = np.array([1.0, 2.0, 1.0])
basis_a = BasisFunction(center_a, lmn_a, exps_a, coefs_a)
center_b = np.array([1.0, 0.0, 0.0])
lmn_b = (2, 1, 1)
exps_b = np.array([1.0, 2.0, 3.0])
coefs_b = np.array([1.0, 2.0, 1.0])
basis_b = BasisFunction(center_b, lmn_b, exps_b, coefs_b)
%time S(basis_a, basis_b)
%time T(basis_a, basis_b)
%time V(basis_a, basis_b, np.array([0.0, 0.0, 0.0]))
%time ERI(basis_a, basis_b, basis_a, basis_b)

CPU times: user 17 µs, sys: 3 µs, total: 20 µs
Wall time: 21.5 µs
CPU times: user 89 µs, sys: 16 µs, total: 105 µs
Wall time: 113 µs
CPU times: user 0 ns, sys: 1.66 ms, total: 1.66 ms
Wall time: 1.39 ms
CPU times: user 802 ms, sys: 18.6 ms, total: 820 ms
Wall time: 820 ms


0.17496143269511694

In [4]:
center_a = np.array([0.0, 0.0, 0.0])
lmn_a = (1, 1, 0)
exps_a = np.array([1.0, 2.0, 3.0])
coefs_a = np.array([1.0, 2.0, 1.0])
basis_a = BasisFunction(center_a, lmn_a, exps_a, coefs_a)
T(basis_a, basis_a)

6.683076777167257